In [2]:
import numpy as np
import torch
import os
from pathlib import Path

### Finding correlation for one probe

In [3]:
p0 = np.load("data_numpy/probe0_data128.npy", allow_pickle=True).item()

p_num = p0['probe_num']
p_pos = p0['position']
p_data = p0['measurement']

print(f"probe number: {p_num}")
print(f"probe position: {p_pos}")
print(f"size of measurement: {len(p_data)}")

probe number: 0
probe position: {'x': 26, 'z': 64}
size of measurement: 7439539


In [4]:
sim_data = np.load("data_numpy/density_large128.npy")

### Retrieving probe points for rolled simulation data

In [5]:
# Find all probe positions
probe_files = ['probe0_data128.npy', 'probe1_data128.npy', 'probe2_data128.npy', 'probe3_data128.npy',
                'probe4_data128.npy', 'probe5_data128.npy', 'probe6_data128.npy']
probe_data = {}
probe_nums = []

for pf in probe_files:
    path = Path('data_numpy/' + pf)
    probe = np.load(path, allow_pickle=True).item()
    probe_data[probe['probe_num']] = probe['position']
    print(f"Probe {probe['probe_num']} loaded with position {probe_data[probe['probe_num']]}")


Probe 0 loaded with position {'x': 26, 'z': 64}
Probe 1 loaded with position {'x': 39, 'z': 64}
Probe 2 loaded with position {'x': 51, 'z': 64}
Probe 3 loaded with position {'x': 64, 'z': 64}
Probe 4 loaded with position {'x': 77, 'z': 64}
Probe 2 loaded with position {'x': 51, 'z': 64}
Probe 3 loaded with position {'x': 64, 'z': 64}
Probe 4 loaded with position {'x': 77, 'z': 64}
Probe 5 loaded with position {'x': 90, 'z': 64}
Probe 6 loaded with position {'x': 103, 'z': 64}
Probe 5 loaded with position {'x': 90, 'z': 64}
Probe 6 loaded with position {'x': 103, 'z': 64}


In [6]:
# If you want to add more probes

# Add two extra lines of probrs at y = 0.75 and y = 0.25
print(probe_data)
k = len(probe_data)
for i in range(k):
    
    new_z_hi = round(probe_data[i]['z'] * 1.5)
    probe_data[i+k] = {'x': probe_data[i]['x'], 'z': new_z_hi}

    new_z_lo = round(probe_data[i]['z'] * 0.5)
    probe_data[i+k*2] = {'x': probe_data[i]['x'], 'z': new_z_lo}


print(probe_data)


{0: {'x': 26, 'z': 64}, 1: {'x': 39, 'z': 64}, 2: {'x': 51, 'z': 64}, 3: {'x': 64, 'z': 64}, 4: {'x': 77, 'z': 64}, 5: {'x': 90, 'z': 64}, 6: {'x': 103, 'z': 64}}
{0: {'x': 26, 'z': 64}, 1: {'x': 39, 'z': 64}, 2: {'x': 51, 'z': 64}, 3: {'x': 64, 'z': 64}, 4: {'x': 77, 'z': 64}, 5: {'x': 90, 'z': 64}, 6: {'x': 103, 'z': 64}, 7: {'x': 26, 'z': 96}, 14: {'x': 26, 'z': 32}, 8: {'x': 39, 'z': 96}, 15: {'x': 39, 'z': 32}, 9: {'x': 51, 'z': 96}, 16: {'x': 51, 'z': 32}, 10: {'x': 64, 'z': 96}, 17: {'x': 64, 'z': 32}, 11: {'x': 77, 'z': 96}, 18: {'x': 77, 'z': 32}, 12: {'x': 90, 'z': 96}, 19: {'x': 90, 'z': 32}, 13: {'x': 103, 'z': 96}, 20: {'x': 103, 'z': 32}}


In [ ]:
# We wish to find each image corresponding point data for each probe
# Load simulation frames. Adjust filename if you prefer the rolled version
sim_data = np.load("data_numpy/density_large128.npy")  # shape: (T, H, W)

# Ensure probe_data is a dict mapping probe_num -> {'x':..., 'z':...}
probe_keys = sorted(probe_data.keys())
P = len(probe_keys)
T = sim_data.shape[0]

# extracted_probe_data: we'll build as (P, T) then transpose to (T, P)
extracted_probe_data = np.zeros((P, T), dtype=np.float32)
probe_positions = np.zeros((P, 3), dtype=np.float32)  # columns: x, z, and we'll add normalized x

for i, k in enumerate(probe_keys):
    coord = probe_data[k]
    x = (coord['x'])
    z = (coord['z'])
    probe_positions[i, 0] = x
    probe_positions[i, 1] = z
    print(f"Extracting probe {k} at (x={x}, z={z})")

    for t in range(T):
        extracted_probe_data[i, t] = sim_data[t, x, z]

# Transpose to (T, P) so samples are first dimension
extracted_probe_data_TP = extracted_probe_data.T  # shape (T, P)

# Keep variable name compatible with downstream cells that expect `extracted_probe_data`
extracted_probe_data = extracted_probe_data_TP

# Save positions for future reference
np.save('data_numpy/probe_positions_3lines.npy', probe_positions)

# ------------------------------------------------------------------
# Build feature matrices that include x and z per probe
# Z and X values are in pixel coordinates, normalize them to [0, 1]
z_coords = probe_positions[:, 1].astype(np.float32) / 128  # normalize by image height
x_coords = probe_positions[:, 0].astype(np.float32) / 128  # normalize by image width

# Expand x and z per sample: shape (T, P)
x_per_sample = np.tile(x_coords.reshape(1, P), (T, 1))
z_per_sample = np.tile(z_coords.reshape(1, P), (T, 1))

# Option A: flat features per sample [m1, x1, z1, m2, x2, z2, ...] -> shape (T, P*3)
features_measurement_plus_xz = np.empty((T, P * 3), dtype=np.float32)
features_measurement_plus_xz[:, 0::3] = extracted_probe_data  # measurements
features_measurement_plus_xz[:, 1::3] = x_per_sample          # x values (normalized 0-1)
features_measurement_plus_xz[:, 2::3] = z_per_sample          # z values (normalized 0-1)
np.save('data_numpy/features_measurement_plus_xz.npy', features_measurement_plus_xz)

print('Extraction complete. Saved probe_positions and feature files with x and z coordinates.')
print(f'features_measurement_plus_xz shape: {features_measurement_plus_xz.shape}')
print(f'X coordinate range: [{x_coords.min():.3f}, {x_coords.max():.3f}]')
print(f'Z coordinate range: [{z_coords.min():.3f}, {z_coords.max():.3f}]')


Extracting probe 0 at (x=26, z=64)
Extracting probe 1 at (x=39, z=64)
Extracting probe 2 at (x=51, z=64)
Extracting probe 3 at (x=64, z=64)
Extracting probe 4 at (x=77, z=64)
Extracting probe 5 at (x=90, z=64)
Extracting probe 6 at (x=103, z=64)
Extracting probe 7 at (x=26, z=96)
Extracting probe 8 at (x=39, z=96)
Extracting probe 9 at (x=51, z=96)
Extracting probe 10 at (x=64, z=96)
Extracting probe 11 at (x=77, z=96)
Extracting probe 12 at (x=90, z=96)
Extracting probe 13 at (x=103, z=96)
Extracting probe 14 at (x=26, z=32)
Extracting probe 15 at (x=39, z=32)
Extracting probe 16 at (x=51, z=32)
Extracting probe 17 at (x=64, z=32)
Extracting probe 18 at (x=77, z=32)
Extracting probe 19 at (x=90, z=32)
Extracting probe 20 at (x=103, z=32)
Extraction complete. Saved probe_positions and feature files with x and z coordinates.
features_measurement_plus_xz shape: (501, 63)
features_grouped_T_P_F_xz shape: (501, 21, 3)
X coordinate range: [0.203, 0.805]
Z coordinate range: [0.250, 0.750]


In [38]:
# We wish to find each image corresponding point data for each probe
sim_data = np.load("data_numpy/density_large128.npy")

# Insert probe_measurements here
extracted_probe_data = np.zeros((7*3,501))

for i in range(len(probe_data)):
    coord = probe_data[i]
    x = coord['x']
    y = coord['z']
    print(f"Extracted points for (x = {x}, y = {y})")

    for j in range(len(probe_data[0])):
        extracted_probe_data[i, j] = sim_data[j, x, y]


Extracted points for (x = 26, y = 64)
Extracted points for (x = 39, y = 64)
Extracted points for (x = 51, y = 64)
Extracted points for (x = 64, y = 64)
Extracted points for (x = 77, y = 64)
Extracted points for (x = 90, y = 64)
Extracted points for (x = 103, y = 64)
Extracted points for (x = 26, y = 96)
Extracted points for (x = 39, y = 96)
Extracted points for (x = 51, y = 96)
Extracted points for (x = 64, y = 96)
Extracted points for (x = 77, y = 96)
Extracted points for (x = 90, y = 96)
Extracted points for (x = 103, y = 96)
Extracted points for (x = 26, y = 32)
Extracted points for (x = 39, y = 32)
Extracted points for (x = 51, y = 32)
Extracted points for (x = 64, y = 32)
Extracted points for (x = 77, y = 32)
Extracted points for (x = 90, y = 32)
Extracted points for (x = 103, y = 32)


In [39]:
# Save the extracted probe data as a single array and per-probe files
file_name = 'extracted_probe_data128_3lines.npy'
destination_path = 'data_numpy/' + file_name
np.save(destination_path, extracted_probe_data)

### Making Extra probe points